In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
# from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [ ]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()
esaScale = esa_land_cover.projection().nominalScale()
builtup = esa_land_cover.eq(50)

In [ ]:
# read Aqueduct Floods data
floodImages = ee.ImageCollection("projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation")
def constYear(img):
    img = ee.Image(img)  
    yearConst = 1980
    return ee.Algorithms.If(img.get('year_string'),img.set({'year':yearConst}),img.set({}))

floodImages = floodImages.map(constYear,False)

  # Shared flooding variables 
minInnundation = 0
    #note that return period 1.5 is only available for certain settings.
returnPeriodC = "rp0100"; # option [  "rp0002",  "rp0005",  "rp0010",  "rp0025", "rp0050",  "rp0100",  "rp0250",  "rp0500",  "rp1000"]
returnPeriodR = "rp00100"; # option [  "rp00002",  "rp00005",  "rp00010",  "rp00025", "rp00050",  "rp00100",  "rp00250",  "rp00500",  "rp01000"]
#year = opts.get('year', 2030); # option [1980,2030,2050,2080] files that had year='hist' have a property called year_string since year is a reseverd keyword in earthengine.
startYear = 1980 #Options: 1980,2030,2050,2080.  If 1980, may need to remove some filters that are not available as properties in those images. 
endYear = 2050 #Options: 1980,2030,2050,2080.  If 1980, may need to remove some filters that are not available as properties in those images. 
startYearStr = str(startYear)
endYearStr = str(endYear)
climate = "rcp8p5"; # options "historical","rcp4p5", "rcp8p5"
  # Coastal flooding variables 
subsidence = 'nosub'; #options "nosub", "wtsub"
sea_level_rise_scenario = 50; # options 5, 50
  # Riverine flooding variables 
model = '0000GFDL-ESM2M';
  
Coastal_start = (floodImages.filterMetadata("floodtype","equals","inuncoast")
                      .filterMetadata("returnperiod","equals",returnPeriodC)
                      .filterMetadata("year","equals",startYear)
                      #.filterMetadata("climate","equals",climate)
                      #.filterMetadata("subsidence","equals",subsidence)
                      #.filterMetadata("sea_level_rise_scenario","equals",sea_level_rise_scenario)
                      .first()
                      #.unmask(ee.Image.constant(0));
                )
Coastal_end = (floodImages.filterMetadata("floodtype","equals","inuncoast")
                      .filterMetadata("returnperiod","equals",returnPeriodC)
                      .filterMetadata("year","equals",endYear)
                      .filterMetadata("climate","equals",climate)
                      .filterMetadata("subsidence","equals",subsidence)
                      .filterMetadata("sea_level_rise_scenario","equals",sea_level_rise_scenario)
                      .first()
                      #.unmask(ee.Image.constant(0));
                  )
imageVisParam = {"opacity":1,"bands":["b1"],"max":2,"palette":["ffffff","00cfff","0013ff"]};
#Map.addLayer(Coastal_end,imageVisParam,"Coastal Flooding",1,0.7)


Riverine_start = (floodImages.filterMetadata("floodtype","equals","inunriver")
                      .filterMetadata("returnperiod","equals",returnPeriodR)
                      .filterMetadata("year","equals",startYear)
                      #.filterMetadata("climate","equals",climate)
                      #.filterMetadata("model", "equals", model)
                      .first()
                      #.unmask(ee.Image.constant(0));
                 )
Riverine_end = (floodImages.filterMetadata("floodtype","equals","inunriver")
                      .filterMetadata("returnperiod","equals",returnPeriodR)
                      .filterMetadata("year","equals",endYear)
                      .filterMetadata("climate","equals",climate)
                      .filterMetadata("model", "equals", model)                      
                      .first()
                      #.unmask(ee.Image.constant(0));
               )
imageVisParam = {"opacity":1,"bands":["b1"],"max":2,"palette":["ffffff","00cfff","0013ff"]};
#Map.addLayer(Riverine_end,imageVisParam,"Riverine Flooding",1,0.7);

Combflood_start = Coastal_start.max(Riverine_start)
Combflood_start = Combflood_start.updateMask(Combflood_start.gt(minInnundation))
Combflood_end = Coastal_end.max(Riverine_end)
Combflood_end = Combflood_end.updateMask(Combflood_end.gt(minInnundation))

# print(Combflood_end)
# Map.addLayer(Combflood_end,imageVisParam,"Combined Flooding",1,0.7);

UrbFlood_start = Combflood_start.updateMask(builtup)
UrbFlood_end = Combflood_end.updateMask(builtup)
# Map.addLayer(UrbFlood,imageVisParam,"Urban Flooding",1,0.7);

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Compute indicator

In [ ]:
this_indicator = pd.DataFrame() 

In [ ]:
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcs(FC,DF):
    # reduce images to get flood and built-up pixel counts
    pixelcounts = UrbFlood_start.reduceRegions(FC,ee.Reducer.count().setOutputs(['FloodPixels_start']),esaScale)
    pixelcounts = UrbFlood_end.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['FloodPixels_end']),esaScale)
    pixelcounts = builtup.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['BuiltPixels']),esaScale)

    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        pct_start = (feat.getNumber('FloodPixels_start').divide(feat.getNumber('BuiltPixels')))#.multiply(100)
        pct_end = (feat.getNumber('FloodPixels_end').divide(feat.getNumber('BuiltPixels')))#.multiply(100)
        return feat.set({
            'PctBuiltwFlood_'+startYearStr+'': pct_start,
            'PctBuiltwFlood_'+endYearStr+'': pct_end
      })

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctBuiltwFlood_'+endYearStr+''])

    # store in df and apend
    df = geemap.ee_to_pandas(pixelcounts)
    df = df.rename(columns={'PctBuiltwFlood_'+endYearStr+'': 'GRE_4_1_percentFloodProneinBuiltup'+endYearStr+''})
    DF = DF.append(df)
    return DF

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
        
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)


In [ ]:
this_indicator

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2.csv') 
cities_indicators#.head()

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace by new indicators")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicator,
                                            indicator_name = 'GRE_4_1_percentFloodProneinBuiltup'+endYearStr+'')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')